<a href="https://colab.research.google.com/github/HNXJ/PredictiveCodingNetwork/blob/main/PredictiveLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from tensorflow.keras import backend as K
from matplotlib import pyplot as plt
import tensorflow as tf
import numpy as np


# GPU config if needed
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Recurrent neural network based on predictive coding properties

In this simulation, we will use task of [Abdullahi et. al 2020] by increasing temporal resolution from one image per step to 10 iterations for each input.

In [4]:
# TB
# dir(tf.keras.layers.LSTM)

In [34]:
def create_serial_dataset(x=None, y=None, n=100, length=10, frames=10):

    X = np.zeros([n, length*frames, x.shape[1], x.shape[2]])
    Y = np.zeros([n, length*frames, 10])

    for i in range(n):

        k = np.random.randint(0, 1000, size=(length))
        for j in range(length):
          
            X[i, j*frames:j*frames+frames, :, :] = x[k[j], :, :]
            Y[i, j*frames:j*frames+frames, y[k[j]]] = 1
          
    return X.reshape(n, length*frames, x.shape[1] * x.shape[2]), Y

def init_MNIST():

    (x_train,y_train), (x_test,y_test) = tf.keras.datasets.mnist.load_data()
    Xn, Yn = create_serial_dataset(x_train, y_train, n=100, length=10, frames=10)
    Xn /= 255.0
    Yn /= 255.0
    X = Xn[:10, :, :]
    Y = Yn[:10, :, :]
    return X, Y, Xn, Yn

In [35]:
class RNNModel1(tf.keras.Model):
    def __init__(self):
        super(RNNModel1, self).__init__()
        self.Input = (tf.keras.layers.InputLayer(input_shape=(None, 784)))
        self.LSTM = tf.keras.layers.LSTM(input_shape=(None, 784),
          units=512,
          recurrent_dropout=0.2,
          return_sequences=True,
          # return_state=True
        )
        self.FCN = tf.keras.layers.Dense(units=10)
        return 
        
    def call(self, x):
        out = self.Input(x)
        out = self.LSTM(x)
        out = self.FCN(out)
        return out

    def get_state(self):
        return

In [36]:
class PredictiveNet():
    def __init__(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.InputLayer(input_shape=(None, 784)))
        self.model.add(tf.keras.layers.LSTM(
          units=512,
          recurrent_dropout=0.2,
          return_sequences=True,
          activation="sigmoid"
          # return_state=True
        ))
        self.model.add(tf.keras.layers.Dense(units=10))
        self.lastPreactivation = None
        return

    def printw(self): # Debug log
        print(K.mean(self.model.layers[1].weights[0]))
        return

    def getPreactivation(self, x):
        MTemp = K.function([self.model.layers[0].input],
                                  [self.model.layers[1].input])
        stateVector = MTemp(x)
        self.lastPreactivation = stateVector[0]
        return self.lastPreactivation

    def EnergyCostLoss(self, y_true, y_pred):
        error = y_pred - y_true
        lambda1 = 1.2
        lambda2 = 0.1
        lambda3 = 0.1
        # preact = self.lastPreactivation
        return K.mean(K.square(error) + lambda1*K.mean(K.abs(y_pred))) + lambda2*K.mean(K.abs(self.model.layers[0].weights[0])) + lambda3*K.mean(K.abs(self.model.layers[1].weights[0]))

In [37]:
X, Y, Xn, Yn = init_MNIST()
Net1 = PredictiveNet()
# a = model.predict(Xn[1:2, :, :])
Net1.model.compile(
  loss=Net1.EnergyCostLoss,
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.003)
)
Net1.model.summary()

Net1.printw()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, None, 512)         2656256   
_________________________________________________________________
dense_1 (Dense)              (None, None, 10)          5130      
Total params: 2,661,386
Trainable params: 2,661,386
Non-trainable params: 0
_________________________________________________________________
tf.Tensor(-0.00035928588, shape=(), dtype=float32)


In [38]:
history = Net1.model.fit(
    x=X, y=Y,
    epochs=100,
    batch_size=5,
    validation_split=0.0,
    verbose=1,
    shuffle=True
)

Epoch 1/100
2/2 [==============================] - 1s 136ms/step - loss: 3.6393
Epoch 2/100
2/2 [==============================] - 0s 132ms/step - loss: 1.4566
Epoch 3/100
2/2 [==============================] - 0s 128ms/step - loss: 0.6455
Epoch 4/100
2/2 [==============================] - 0s 130ms/step - loss: 0.9001
Epoch 5/100
2/2 [==============================] - 0s 126ms/step - loss: 0.6212
Epoch 6/100
2/2 [==============================] - 0s 137ms/step - loss: 0.3077
Epoch 7/100
2/2 [==============================] - 0s 132ms/step - loss: 0.1503
Epoch 8/100
2/2 [==============================] - 0s 130ms/step - loss: 0.1477
Epoch 9/100
2/2 [==============================] - 0s 141ms/step - loss: 0.1884
Epoch 10/100
2/2 [==============================] - 0s 136ms/step - loss: 0.1920
Epoch 11/100
2/2 [==============================] - 0s 138ms/step - loss: 0.1581
Epoch 12/100
2/2 [==============================] - 0s 133ms/step - loss: 0.1135
Epoch 13/100
2/2 [===================

In [39]:
# Single epoch for test
history = Net1.model.fit(
    x=X, y=Y,
    epochs=1,
    batch_size=5,
    validation_split=0.0,
    verbose=1,
    shuffle=True
)
# history2 = L1.fit(x=X, y=Y, epochs=1, batch_size=5, validation_split=0.0, verbose=1, shuffle=True)

2/2 [==============================] - 0s 144ms/step - loss: 0.0132


In [40]:
Net1.getPreactivation(Xn[8:9, :, :]).shape

(1, 100, 512)

In [41]:
means1 = []
means1a = []

for k in range(10):
    l, r = k, k+1
    l_pred = Net1.getPreactivation(Xn[l:r, :, :])
    y_pred = Net1.model.predict(Xn[l:r, :, :])

    # for i in range(7):
        # plt.plot(l_pred[0][0, :, i])
    # for i in range(10):
    # plt.plot(y_pred[0, :, 6])
    # for i in range(0, 100, 10):
    #     print(Yn[l:r, i, :])
    means1.append(np.mean(np.mean(l_pred)))
    means1a.append(np.mean(np.mean(np.abs(l_pred))))

In [42]:
means2 = []
means2a = []

for k in range(10, 50):
    l, r = k, k+1
    l_pred = Net1.getPreactivation(Xn[l:r, :, :])
    y_pred = Net1.model.predict(Xn[l:r, :, :])

    # for i in range(7):
    #     plt.plot(l_pred[0, i, :])
    # for i in range(10):
    # plt.plot(y_pred[0, :, 6])
    # for i in range(0, 100, 10):
    #     print(Yn[l:r, i, :])
    means2.append(np.mean(np.mean(l_pred)))
    means2a.append(np.mean(np.mean(np.abs(l_pred))))

In [47]:
print("Mean energy on recurrent preactivation (LSTM) on sigmoid activation function")
print(np.mean(means1), np.mean(means2), "(ME)")
print(np.mean(means1a), np.mean(means2a), "(MAE)")
print(100*np.abs(np.mean(means1)-np.mean(means2))/np.mean(means2), "(Difference ratio, percentage %)")

Mean energy on recurrent preactivation (LSTM) on sigmoid activation function
0.050481953 0.052655827 (ME)
0.050481953 0.052655827 (MAE)
4.128459546798724 (Difference ratio, percentage %)


## Lesioning the network




In [49]:
w = Net1.model.get_weights()
for i in range(len(w)):
    w[i] += (np.random.rand(*w[i].shape)-0.5)*0.02
Net1.model.set_weights(w)

In [50]:
means1 = []

for k in range(10):
    l, r = k, k+1
    l_pred = Net1.getPreactivation(Xn[l:r, :, :])[0]
    y_pred = Net1.model.predict(Xn[l:r, :, :])

    # for i in range(7):
        # plt.plot(l_pred[0][0, :, i])
    # for i in range(10):
    # plt.plot(y_pred[0, :, 6])
    # for i in range(0, 100, 10):
    #     print(Yn[l:r, i, :])
    means1.append(np.mean(np.mean(l_pred)))

In [51]:
means2 = []

for k in range(10, 50):
    l, r = k, k+1
    l_pred = Net1.getPreactivation(Xn[l:r, :, :])[0]
    y_pred = Net1.model.predict(Xn[l:r, :, :])

    # for i in range(7):
    #     plt.plot(l_pred[0, i, :])
    # for i in range(10):
    # plt.plot(y_pred[0, :, 6])
    # for i in range(0, 100, 10):
    #     print(Yn[l:r, i, :])
    means2.append(np.mean(np.mean(l_pred)))

In [52]:
print(np.mean(means1), np.mean(means2))

23.622051 23.088428
